Сентемент анализ корпуса текстов

In [ ]:
import os
import time
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.layers import (Input, Embedding, LSTM, Bidirectional, Dense, Dropout,
                                     SpatialDropout1D, GlobalMaxPooling1D, Attention)
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant

In [ ]:
max_features = 20000
maxlen = 200
embedding_dim = 100
batch_size = 64
epochs = 20

In [ ]:
print("IMDB...")
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

print("Train shape:", x_train.shape, "Test shape:", x_test.shape)

if not os.path.exists("glove.6B.100d.txt"):
    !wget http://nlp.stanford.edu/data/glove.6B.zip
    !unzip -q glove.6B.zip

IMDB...
Train shape: (25000, 200) Test shape: (25000, 200)


In [ ]:
word_index = imdb.get_word_index()
reverse_word_index = {value+3: key for (key, value) in word_index.items()}
reverse_word_index[0] = "<PAD>"
reverse_word_index[1] = "<START>"
reverse_word_index[2] = "<UNK>"

In [ ]:
print("GloVe...")
embedding_index = {}
with open("glove.6B.100d.txt", encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype="float32")
        embedding_index[word] = vector


print(f"Загружено векторов: {len(embedding_index)}")


embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i < max_features:
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

GloVe...
Загружено векторов: 400000


In [ ]:
def build_model():
    inputs = Input(shape=(maxlen,))
    x = Embedding(
        input_dim=max_features,
        output_dim=embedding_dim,
        embeddings_initializer=Constant(embedding_matrix),
        input_length=maxlen,
        trainable=False
    )(inputs)

    x = SpatialDropout1D(0.25)(x)
    x = Bidirectional(LSTM(128, return_sequences=True))(x)

    attention = Attention()([x, x])
    x = tf.keras.layers.Add()([x, attention])  # residual connection

    x = GlobalMaxPooling1D()(x)
    x = Dense(64, activation="relu")(x)
    x = Dropout(0.3)(x)
    outputs = Dense(1, activation="sigmoid")(x)

    model = Model(inputs, outputs)
    model.compile(loss="binary_crossentropy",
                  optimizer="adam",
                  metrics=["accuracy"])
    return model

In [ ]:
device = "GPU" if tf.config.list_physical_devices("GPU") else "CPU"

model = build_model()
model.summary()

start = time.time()
history = model.fit(
    x_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(x_test, y_test),
    verbose=1
)

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1       │ (None, 200)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 200, 100)  │  2,000,000 │ input_layer_1[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ spatial_dropout1d_1 │ (None, 200, 100)  │          0 │ embedding_1[0][0] │
│ (SpatialDropout1D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional_1     │ (None, 200, 256)  │    234,496 │ spatial_dropout1… │
│ (Bidirectional)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_1         │ (None, 200, 256)  │          0 │ bidirectional_1[… │
│ (Attention)         │                   │            │ bidirectional_1[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 200, 256)  │          0 │ bidirectional_1[… │
│                     │                   │            │ attention_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_max_pooling… │ (None, 256)       │          0 │ add_1[0][0]       │
│ (GlobalMaxPooling1… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │     16,448 │ global_max_pooli… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64)        │          0 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         65 │ dropout_1[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,251,009 (8.59 MB)

 Trainable params: 251,009 (980.50 KB)

 Non-trainable params: 2,000,000 (7.63 MB)

Epoch 1/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 19s 43ms/step - accuracy: 0.5132 - loss: 0.6978 - val_accuracy: 0.5690 - val_loss: 0.6903
Epoch 2/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.5486 - loss: 0.6850 - val_accuracy: 0.6471 - val_loss: 0.6255
Epoch 3/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.6721 - loss: 0.6030 - val_accuracy: 0.7517 - val_loss: 0.5074
Epoch 4/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 39ms/step - accuracy: 0.7537 - loss: 0.5021 - val_accuracy: 0.7863 - val_loss: 0.4520
Epoch 5/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.8027 - loss: 0.4358 - val_accuracy: 0.8090 - val_loss: 0.4151
Epoch 6/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 20s 41ms/step - accuracy: 0.8271 - loss: 0.3988 - val_accuracy: 0.8245 - val_loss: 0.3801
Epoch 7/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 21s 41ms/step - accuracy: 0.8418 - loss: 0.3616 - val_accuracy: 0.8423 - val_loss: 0.3525
Epoch 8/20
391/391 ━━━━━━━━━━━━━━━━━━━━ 16s 41ms/step - accuracy: 0.8613 - loss: 0.3283 - 

In [ ]:
loss, acc = model.evaluate(x_test, y_test, verbose=0)
print(f"Точность на тесте: {acc*100:.2f}%")

Точность на тесте: 86.34%
